In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 0
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC50(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC50(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC50(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料
讀取3秒資料
讀取5秒資料
Data Loading Time: 0.28 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []


#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    #------------------------------------------------
    #需要調整的地方
    #------------------------------------------------
    #學習率
    lr= 0.00032
    #epoch
    epoch= 120
    #訓練資料(要用幾秒訓練) data_1S/data_3S/data_5S
    trainData = data_1S
    #------------------------------------------------
    
    #紀錄訓練開始時間
    start_time = time.time()

    #訓練模型
    model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold,
                                                                setLR=lr,
                                                                setEpoch=epoch)
    
    #測試準確度
    test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
    test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
    test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

    #紀錄訓練結束時間
    end_time = time.time()

    #計算訓練耗時
    training_time = end_time - start_time

    #印出訓練耗時
    print("Training Time: {:.2f} seconds".format(training_time))

    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("訓練過程\n")
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    for i in range(len(train_loss)):
        # 寫入資料
        f.write("Fold: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
    f.write("Training Time: {:.2f} seconds\n".format(training_time))
    f.write("---------------------------------------------------------------------------------------------------------------\n")

    #--------------------------------------------------------------------

    
    kFoldTrainTime.append(training_time)
    kFoldTrainAccuracy.append(train_accuracy[-1])
    kFoldTestAccuracy1.append(test_accuracy1)
    kFoldTestAccuracy3.append(test_accuracy3)
    kFoldTestAccuracy5.append(test_accuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
可用的 GPU 数量: 1
Train Epoch: 1/120 Train_Loss: 1.8193050622940063 Train_acc: 30.100002%
Train Epoch: 2/120 Train_Loss: 1.2028123140335083 Train_acc: 56.525002%
Train Epoch: 3/120 Train_Loss: 1.2143038511276245 Train_acc: 70.612503%
Train Epoch: 4/120 Train_Loss: 0.9347198009490967 Train_acc: 77.900002%
Train Epoch: 5/120 Train_Loss: 0.7057081460952759 Train_acc: 82.800003%
Train Epoch: 6/120 Train_Loss: 0.5207677483558655 Train_acc: 84.837502%
Train Epoch: 7/120 Train_Loss: 0.4722791910171509 Train_acc: 87.550003%
Train Epoch: 8/120 Train_Loss: 0.6154981851577759 Train_acc: 88.237503%
Train Epoch: 9/120 Train_Loss: 0.4809061884880066 Train_acc: 88.325005%
Train Epoch: 10/120 Train_Loss: 0.422564297914505 Train_acc: 89.662506%
Train Epoch: 11/120 Train_Loss:

Train Epoch: 110/120 Train_Loss: 0.1893262267112732 Train_acc: 93.187508%
Train Epoch: 111/120 Train_Loss: 0.1714247316122055 Train_acc: 93.225006%
Train Epoch: 112/120 Train_Loss: 0.005696794483810663 Train_acc: 93.237503%
Train Epoch: 113/120 Train_Loss: 0.20508091151714325 Train_acc: 93.237503%
Train Epoch: 114/120 Train_Loss: 0.04754521697759628 Train_acc: 93.237503%
Train Epoch: 115/120 Train_Loss: 0.132563054561615 Train_acc: 93.225006%
Train Epoch: 116/120 Train_Loss: 0.27663832902908325 Train_acc: 93.225006%
Train Epoch: 117/120 Train_Loss: 0.12196516245603561 Train_acc: 93.237503%
Train Epoch: 118/120 Train_Loss: 0.07956913858652115 Train_acc: 93.275002%
Train Epoch: 119/120 Train_Loss: 0.2821982204914093 Train_acc: 93.237503%
Train Epoch: 120/120 Train_Loss: 0.22453486919403076 Train_acc: 93.250008%
Test accuracy of the model: 49.050000%
Test accuracy of the model: 53.666667%
Test accuracy of the model: 58.250000%
Training Time: 343.56 seconds
--------------------------------

Train Epoch: 97/120 Train_Loss: 0.18758352100849152 Train_acc: 93.125008%
Train Epoch: 98/120 Train_Loss: 0.15790137648582458 Train_acc: 93.112503%
Train Epoch: 99/120 Train_Loss: 0.48067259788513184 Train_acc: 93.125008%
Train Epoch: 100/120 Train_Loss: 0.21454264223575592 Train_acc: 93.125008%
Train Epoch: 101/120 Train_Loss: 0.3341187834739685 Train_acc: 93.112503%
Train Epoch: 102/120 Train_Loss: 0.2848975956439972 Train_acc: 93.112503%
Train Epoch: 103/120 Train_Loss: 0.4036361873149872 Train_acc: 93.125008%
Train Epoch: 104/120 Train_Loss: 0.1480487585067749 Train_acc: 93.125008%
Train Epoch: 105/120 Train_Loss: 0.2209486961364746 Train_acc: 93.112503%
Train Epoch: 106/120 Train_Loss: 0.36311212182044983 Train_acc: 93.125008%
Train Epoch: 107/120 Train_Loss: 0.22580362856388092 Train_acc: 93.125008%
Train Epoch: 108/120 Train_Loss: 0.30566972494125366 Train_acc: 93.112503%
Train Epoch: 109/120 Train_Loss: 0.42851072549819946 Train_acc: 93.125008%
Train Epoch: 110/120 Train_Loss: 

Train Epoch: 83/120 Train_Loss: 0.1420515775680542 Train_acc: 93.037506%
Train Epoch: 84/120 Train_Loss: 0.27277252078056335 Train_acc: 93.162506%
Train Epoch: 85/120 Train_Loss: 0.44917184114456177 Train_acc: 93.225006%
Train Epoch: 86/120 Train_Loss: 0.2988816499710083 Train_acc: 93.237503%
Train Epoch: 87/120 Train_Loss: 0.18847358226776123 Train_acc: 93.200005%
Train Epoch: 88/120 Train_Loss: 0.38177943229675293 Train_acc: 93.225006%
Train Epoch: 89/120 Train_Loss: 0.24562808871269226 Train_acc: 93.187508%
Train Epoch: 90/120 Train_Loss: 0.300332635641098 Train_acc: 93.237503%
Train Epoch: 91/120 Train_Loss: 0.15518707036972046 Train_acc: 93.225006%
Train Epoch: 92/120 Train_Loss: 0.5188214778900146 Train_acc: 93.212502%
Train Epoch: 93/120 Train_Loss: 0.21106195449829102 Train_acc: 93.150002%
Train Epoch: 94/120 Train_Loss: 0.18660175800323486 Train_acc: 93.237503%
Train Epoch: 95/120 Train_Loss: 0.23071344196796417 Train_acc: 93.225006%
Train Epoch: 96/120 Train_Loss: 0.048513058

Train Epoch: 69/120 Train_Loss: 0.32707124948501587 Train_acc: 92.750008%
Train Epoch: 70/120 Train_Loss: 0.08956513553857803 Train_acc: 92.787506%
Train Epoch: 71/120 Train_Loss: 0.17045509815216064 Train_acc: 92.650002%
Train Epoch: 72/120 Train_Loss: 0.32629549503326416 Train_acc: 92.725006%
Train Epoch: 73/120 Train_Loss: 0.35796070098876953 Train_acc: 92.837502%
Train Epoch: 74/120 Train_Loss: 0.23032057285308838 Train_acc: 92.562508%
Train Epoch: 75/120 Train_Loss: 0.2072712779045105 Train_acc: 92.737503%
Train Epoch: 76/120 Train_Loss: 0.2720072567462921 Train_acc: 92.787506%
Train Epoch: 77/120 Train_Loss: 0.3552509844303131 Train_acc: 91.800003%
Train Epoch: 78/120 Train_Loss: 0.31504079699516296 Train_acc: 91.975006%
Train Epoch: 79/120 Train_Loss: 0.21581856906414032 Train_acc: 92.550003%
Train Epoch: 80/120 Train_Loss: 0.2280585914850235 Train_acc: 92.762505%
Train Epoch: 81/120 Train_Loss: 0.2213159203529358 Train_acc: 92.725006%
Train Epoch: 82/120 Train_Loss: 0.321893066

Train Epoch: 56/120 Train_Loss: 0.18754060566425323 Train_acc: 92.750008%
Train Epoch: 57/120 Train_Loss: 0.21955394744873047 Train_acc: 92.700005%
Train Epoch: 58/120 Train_Loss: 0.12196963280439377 Train_acc: 92.862503%
Train Epoch: 59/120 Train_Loss: 0.3556046187877655 Train_acc: 92.737503%
Train Epoch: 60/120 Train_Loss: 0.38652563095092773 Train_acc: 92.825005%
Train Epoch: 61/120 Train_Loss: 0.42892298102378845 Train_acc: 92.837502%
Train Epoch: 62/120 Train_Loss: 0.26202598214149475 Train_acc: 92.737503%
Train Epoch: 63/120 Train_Loss: 0.20316089689731598 Train_acc: 92.775002%
Train Epoch: 64/120 Train_Loss: 0.17109781503677368 Train_acc: 92.837502%
Train Epoch: 65/120 Train_Loss: 0.30155491828918457 Train_acc: 92.912506%
Train Epoch: 66/120 Train_Loss: 0.27081698179244995 Train_acc: 92.737503%
Train Epoch: 67/120 Train_Loss: 0.2549244165420532 Train_acc: 92.712502%
Train Epoch: 68/120 Train_Loss: 0.19590023159980774 Train_acc: 92.712502%
Train Epoch: 69/120 Train_Loss: 0.222573

In [5]:
#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    # 寫入資料
    f.write("Fold: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, kFoldTrainAccuracy[fold], kFoldTestAccuracy1[fold], kFoldTestAccuracy3[fold], kFoldTestAccuracy5[fold], kFoldTrainTime[fold]))
    f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(max(kFoldTrainAccuracy), max(kFoldTestAccuracy1), max(kFoldTestAccuracy3), max(kFoldTestAccuracy5)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(min(kFoldTrainAccuracy), min(kFoldTestAccuracy1), min(kFoldTestAccuracy3), min(kFoldTestAccuracy5)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(np.mean(kFoldTrainAccuracy), np.mean(kFoldTestAccuracy1), np.mean(kFoldTestAccuracy3), np.mean(kFoldTestAccuracy5)))

f.close()